https://t.me/ASLRec_bot

In [ ]:
!pip install python-telegram-bot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.7/662.7 kB 16.8 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes
import asyncio
import nest_asyncio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
nest_asyncio.apply()

model = None
labels = ["A", "B", "C"]

def load_asl_model():
    """Функция для загрузки модели (загружается только один раз)."""
    global model
    if model is None:
        model = load_model('/content/drive/MyDrive/ИИ/asl_model.h5')
    return model

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text('Привет! Отправь мне изображение, и я определю жест ASL.')

async def handle_image(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Обработчик фотографий, отправленных пользователем."""
    try:
        if update.message.photo:
            photo = update.message.photo[-1]
            file = await photo.get_file()
            file_path = "received_image.jpg"
            await file.download_to_drive(file_path)

            image = cv2.imread(file_path)
            if image is None:
                await update.message.reply_text('Ошибка: не удалось загрузить изображение.')
                return

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (64, 64))
            image = image / 255.0
            image = np.expand_dims(image, axis=0)

            model = load_asl_model()
            predictions = model.predict(image)
            predicted_label = labels[np.argmax(predictions)]

            await update.message.reply_text(f'Распознанный жест: {predicted_label}')

            os.remove(file_path)
        else:
            await update.message.reply_text('Пожалуйста, отправьте изображение.')
    except Exception as e:
        await update.message.reply_text(f'Произошла ошибка: {e}')

def main():
    TOKEN = '8026100947:AAEJBG5viyG7nHiVbOpqPuQoKX-5CVhMdtw'

    application = Application.builder().token(TOKEN).build()

    application.add_handler(CommandHandler('start', start))
    application.add_handler(MessageHandler(filters.PHOTO, handle_image))

    application.run_polling()

# Запуск бота
if __name__ == '__main__':
    try:
        asyncio.run(main())
    except RuntimeError:
        loop = asyncio.get_event_loop()
        loop.run_until_complete(main())

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
